In [38]:
import json
import math

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm
from datetime import datetime
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler


In [48]:
def ret(n, n_1):
    return (n_1 - n) / n


PREDICTIONS_TRAINING = 1000
PREDICTIONS_TESTING = 700

# prices every 15 seconds
# df = pd.read_csv("prices.csv")
f = open("BTC_USDT-3m.json")
df = pd.DataFrame.from_records(json.load(f), columns=['timestamp', 'open', 'high', 'low', 'close', 'vol'])
df['ret_c'] = (df['close'] - df['close'].shift(1)) / df['close'].shift(1)
df['ret_o'] = (df['open'] - df['open'].shift(1)) / df['open'].shift(1)
df['ret_h'] = (df['high'] - df['high'].shift(1)) / df['high'].shift(1)
df['ret_l'] = (df['low'] - df['low'].shift(1)) / df['low'].shift(1)

df['ret'] = (df['open'].shift(-1) - df['open']) / df['open']

df['ret_s'] = df['ret'].apply(lambda x: np.sign(x))


In [49]:
df

,timestamp,open,high,low,close,vol,ret_c,ret_o,ret_h,ret_l,ret,ret_s
0,1661904000000,19813.03,19838.84,19811.14,19819.50,582.13461,NaN,NaN,NaN,NaN,0.000401,1.0
1,1661904180000,19820.97,19876.77,19797.94,19863.03,605.82699,0.002196,0.000401,0.001912,-0.000666,0.002223,1.0
2,1661904360000,19865.04,19910.78,19861.16,19904.99,578.33857,0.002112,0.002223,0.001711,0.003193,0.002052,1.0
3,1661904540000,19905.80,19910.18,19864.75,19873.44,607.35949,-0.001585,0.002052,-0.000030,0.000181,-0.001626,-1.0
4,1661904720000,19873.44,19892.60,19863.52,19883.66,354.87011,0.000514,-0.001626,-0.000883,-0.000062,0.000466,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5243,1662847740000,21695.72,21747.90,21675.87,21725.07,914.47974,0.001353,-0.000084,0.001325,0.000476,0.001449,1.0
5244,1662847920000,21727.16,21735.53,21690.00,21723.08,710.86427,-0.000092,0.001449,-0.000569,0.000652,-0.000237,-1.0
5245,1662848100000,21722.02,21729.23,21663.22,21689.83,844.59582,-0.001531,-0.000237,-0.000290,-0.001235,-0.001482,-1.0
5246,1662848280000,21689.83,21697.99,21653.75,21658.57,746.62426,-0.001441,-0.001482,-0.001438,-0.000437,-0.001379,-1.0


In [54]:
# training
N_CANDLES = 10
NDIM = N_CANDLES * 4  # NDTR velas * 4 datos por velas
# K, NDTR, hit_rate


In [ ]:
hit_rates_K = []
# K, NDTR, mean(ret)
rets_K = []
for K in tqdm(np.arange(5, 150, 5)):
    for NDTR in (np.arange(150, 4000, 50)):

        knn = KNeighborsClassifier(n_neighbors=K, weights="distance")

        xs = []
        y_train = []
        y_test = []
        y_predicted = []
        for j in np.arange(K + 1 + N_CANDLES, K + 1 + N_CANDLES + NDTR):
            small_df = df.iloc[j - N_CANDLES: j].copy(deep=True)[
                ['ret_o', 'ret_c', 'ret_h', 'ret_l', 'ret_s']].reset_index(
                drop=True)

            x = np.array(small_df[['ret_c', 'ret_h', 'ret_l', 'ret_o']].values).reshape(NDIM)
            y = df.iloc[j]['ret_s']
            xs.append(x)
            y_train.append(y)

        for j in np.arange(K + 1 + NDTR + N_CANDLES, K + 1 + NDTR + N_CANDLES + PREDICTIONS_TESTING):
            small_df = df.iloc[j - N_CANDLES: j].copy(deep=True)[
                ['ret_o', 'ret_c', 'ret_h', 'ret_l', 'ret_s']].reset_index(
                drop=True)
            x = np.array(small_df[['ret_c', 'ret_h', 'ret_l', 'ret_o']].values).reshape(NDIM)
            xs.append(x)
            y_test.append(df.iloc[j]['ret_s'])

        st_x = RobustScaler()
        xs_norm = st_x.fit_transform(xs)
        x_train = xs_norm[:NDTR]
        x_test = xs_norm[NDTR:]
        knn.fit(x_train, y_train)

        predictions = []
        actuals = []
        hits = 0
        rets = []

        for j in np.arange(K + 1 + NDTR + N_CANDLES, K + 1 + NDTR + N_CANDLES + PREDICTIONS_TESTING):
            small_df = df.iloc[j - N_CANDLES: j]

            x_test = [np.array(small_df[['ret_c', 'ret_h', 'ret_l', 'ret_o']].values).reshape(NDIM)]
            y_test = df.iloc[j - 1]['ret_s']
            # st_x= RobustScaler()
            # x_test= st_x.fit_transform(x_test)
            prediction = knn.predict(x_test)[0]
            actual = y_test
            predictions.append(prediction)
            actuals.append(actual)
            if actual == prediction:
                hits += 1
            if prediction > 0:
                rets.append(df.iloc[j - 1]['ret'])
        hit_rates_K.append([K, NDTR, hits / PREDICTIONS_TESTING])
        rets_K.append([K, NDTR, np.array(rets).mean() if len(rets)>0 else 0])


 38%|███▊      | 11/29 [1:10:32<1:57:42, 392.38s/it]

In [52]:
hit_rates_K = pd.DataFrame(hit_rates_K, columns=["K", "NDTR", "hit_rate"])
rets_K = pd.DataFrame(rets_K, columns=["K", "NDTR", "ret"])
print(hit_rates_K['hit_rate'].max())
print(rets_K['ret'].max())

0.6085714285714285
0.0034869021597492356


In [53]:
hit_rates_K.to_csv("knn/first_run_knn_hr_700testing_10candles.csv")
rets_K.to_csv("knn/first_run_knn_ret_700testing_10candles.csv")